# Офлайн метрика

Сейчас мы попробуем создать простейшую офлайн метрику.

Наш сервис – новостное интернет издание The Guardian. Мы хотим сделать метрику популярности статей про футбол. В качестве источника разметки будем использовать поисковую выдачу.

#### Данные

Возьмем набор общих запросов про футбольные клубы и продублируем их на разных языках. И посмотрим, будет ли в выдаче наш сайт.

Будем считать, что нам уже отобрали футбольные клубы и перевели их на несколько языков. Источник данных на Kaggle https://www.kaggle.com/eliasdabbas/european-football-soccer-clubs-on-google-serps

In [1]:
import pandas as pd
from functools import reduce

In [2]:
serp_clubs = pd.read_csv('football_clubs_search_data/serp_clubs.csv')

In [3]:
serp_clubs.head(10)

,hl,searchTerms,rank,title,snippet,displayLink,link,queryTime,totalResults,cacheId,...,htmlTitle,inputEncoding,kind,outputEncoding,pagemap,safe,searchTime,startIndex,fileFormat,mime
0,en,real madrid football,1,Real Madrid CF | Official Website,"Real Madrid official website with news, photos...",www.realmadrid.com,https://www.realmadrid.com/en,2019-07-12 12:00:17.776752+00:00,309000000,ufkNert3UXcJ,...,<b>Real Madrid</b> CF | Official Website,utf8,customsearch#result,utf8,"{'cse_thumbnail': [{'width': '345', 'height': ...",off,0.365675,1,NaN,NaN
1,en,real madrid football,2,Real Madrid CF - Wikipedia,Real Madrid Club de Fútbol commonly referred t...,en.wikipedia.org,https://en.wikipedia.org/wiki/Real_Madrid_CF,2019-07-12 12:00:17.776752+00:00,309000000,8h4sB2TyUxkJ,...,<b>Real Madrid</b> CF - Wikipedia,utf8,customsearch#result,utf8,"{'cse_thumbnail': [{'width': '130', 'height': ...",off,0.365675,1,NaN,NaN
2,en,real madrid football,3,Real Madrid Football and News | Real Madrid CF,News. Militão's presentation at the Santiago B...,www.realmadrid.com,https://www.realmadrid.com/en/football,2019-07-12 12:00:17.776752+00:00,309000000,U6M06aIEUXYJ,...,<b>Real Madrid Football</b> and News | Real Ma...,utf8,customsearch#result,utf8,"{'cse_thumbnail': [{'width': '150', 'height': ...",off,0.365675,1,NaN,NaN
3,en,real madrid football,4,Real Madrid,"Compet. Partido, Resultado. Liga, R. Madrid-Be...",www.marca.com,https://www.marca.com/en/football/real-madrid....,2019-07-12 12:00:17.776752+00:00,309000000,Is7S0CX_fmUJ,...,<b>Real Madrid</b>,utf8,customsearch#result,utf8,"{'cse_thumbnail': [{'width': '298', 'height': ...",off,0.365675,1,NaN,NaN
4,en,real madrid football,5,Real Madrid News | Los Merengues News - NewsNow,"Breaking news headlines about Real Madrid, lin...",www.newsnow.co.uk,https://www.newsnow.co.uk/h/Sport/Football/La+...,2019-07-12 12:00:17.776752+00:00,309000000,XWS-tFRPECoJ,...,Real Madrid News | Los Merengues News - NewsNow,utf8,customsearch#result,utf8,"{'cse_thumbnail': [{'width': '265', 'height': ...",off,0.365675,1,NaN,NaN
5,en,real madrid football,6,Éder Militão falls ill during Real Madrid unve...,1 day ago ... Real Madrid's new signing Éder M...,www.theguardian.com,https://www.theguardian.com/football/video/201...,2019-07-12 12:00:17.776752+00:00,309000000,Osp1Ejbj2ukJ,...,Éder Militão falls ill during <b>Real Madrid</...,utf8,customsearch#result,utf8,{'thumbnail': [{'src': 'https://i.guim.co.uk/i...,off,0.365675,1,NaN,NaN
6,en,real madrid football,7,Real Madrid - Football - BBC Sport,The home of Real Madrid Football on BBC Sport ...,www.bbc.com,https://www.bbc.com/sport/football/teams/real-...,2019-07-12 12:00:17.776752+00:00,309000000,lQ_1KU3V8_MJ,...,<b>Real Madrid</b> - <b>Football</b> - BBC Sport,utf8,customsearch#result,utf8,"{'cse_thumbnail': [{'width': '225', 'height': ...",off,0.365675,1,NaN,NaN
7,en,real madrid football,8,Real Madrid - Football Statistics | WhoScored.com,Real Madrid (Spain) statistics – Check out det...,www.whoscored.com,https://www.whoscored.com/Teams/52/Show/Spain-...,2019-07-12 12:00:17.776752+00:00,309000000,_tRI5GqrQq8J,...,<b>Real Madrid</b> - <b>Football</b> Statistic...,utf8,customsearch#result,utf8,{'metatags': [{'apple-itunes-app': 'app-id=940...,off,0.365675,1,NaN,NaN
8,en,real madrid football,9,Real Madrid - Sky Sports Football,Real Madrid Football Club - get the latest new...,www.skysports.com,https://www.skysports.com/real-madrid,2019-07-12 12:00:17.776752+00:00,309000000,mWhQnS1edVcJ,...,<b>Real Madrid</b> - Sky Sports <b>Football</b>,utf8,customsearch#result,utf8,"{'cse_thumbnail': [{'width': '300', 'height': ...",off,0.365675,1,NaN,NaN
9,en,real madrid football,10,"Real Madrid - Latest news, transfer gossip and...",The most successful team in European Cup and C...,www.mirror.co.uk,https://www.mirror.co.uk/all-about/real-madrid-cf,2019-07-12 12:00:17.776752+00:00,309000000,NaN,...,"<b>Real Madrid</b> - Latest news, transfer gos...",utf8,customsearch#result,utf8,{'newsmediaorga

Посмотрим на результаты с нашего сайта – The Guardian.

In [4]:
serp_clubs[serp_clubs.displayLink == "www.theguardian.com"].head(2)

,hl,searchTerms,rank,title,snippet,displayLink,link,queryTime,totalResults,cacheId,...,htmlTitle,inputEncoding,kind,outputEncoding,pagemap,safe,searchTime,startIndex,fileFormat,mime
5,en,real madrid football,6,Éder Militão falls ill during Real Madrid unve...,1 day ago ... Real Madrid's new signing Éder M...,www.theguardian.com,https://www.theguardian.com/football/video/201...,2019-07-12 12:00:17.776752+00:00,309000000,Osp1Ejbj2ukJ,...,Éder Militão falls ill during <b>Real Madrid</...,utf8,customsearch#result,utf8,{'thumbnail': [{'src': 'https://i.guim.co.uk/i...,off,0.365675,1,NaN,NaN
85,en,atlético madrid football,6,Atlético Madrid open disciplinary proceedings ...,3 days ago ... Atlético Madrid have opened dis...,www.theguardian.com,https://www.theguardian.com/football/2019/jul/...,2019-07-12 12:00:17.859058+00:00,127000000,LHmhiO_ioCQJ,...,<b>Atlético Madrid</b> open disciplinary proce...,utf8,customsearch#result,utf8,{'thumbnail': [{'src': 'https://i.guim.co.uk/i...,off,0.401538,1,NaN,NaN


In [5]:
def check_existence(series):
    return reduce(lambda x, y: x + (1 if y == "www.theguardian.com" else 0), series, 0)

In [6]:
serp_clubs.groupby(["searchTerms", "hl"]).agg({'displayLink': check_existence}).displayLink.sum()

38

####  Как можно улучшить метрику

На самом деле мы много не учли.
* Позицию документа в выдаче.
* 10 из 10 документов с нашего сайта это не обязательно хороший результат, хорошо бы показать хотя бы один из 10, а добавление остальных вносит не такой большой вес.
* А еще хорошо бы сравниться с самым популярным доменом по этим запросам, чтобы понимать, насколько отстаем от потенциального конкурента (хотя это может быть даже не новостной ресурс).

# Домашнее задание

Попробуем улучшить наши метрики. Далее представлены 3 отдельных задания. На каждое из них будет свой ответ. Запишите эти ответы в выходной файл **results.tsv** по одному значению на строку.

In [96]:
RESULT = []

#### Часть 1

Вместо того, чтобы подсчитывать просто вхождения нужного ресурса в выдачу, давайте посчитаем, насколько высоко в каждой из них показываются нужные документы.

В каждой паре **(запрос, регион)** вычислим минимальный ранк (начиная с 1, как в данных) документа с www.theguardian.com. Если такого документа нет, берем значение по-умолчанию – **11**, как будто документ попал на вторую страницу и мы не надеемся, что пользователи его увидят. Чем этот показатель меньше, тем лучше.

Сама метрика – сумма таких показателей по всем имеющимся парам. Полученное значение нужно записать в первую строчку выходного файла.

In [52]:
MAX_DEFAULT_RANK = 11

In [74]:
from functools import partial

def calc_source_rank(series: pd.Series, source: str) -> pd.Series:
    ranks = series[series['displayLink'] == source]['rank'].values
    min_rank = min(ranks) if len(ranks) else MAX_DEFAULT_RANK
    return pd.Series({'source_rank' : min_rank}, index=['source_rank'])

calc_guardian_rank = partial(calc_source_rank, source='www.theguardian.com')

In [75]:
serp_clubs.groupby(['searchTerms', 'hl']).apply(calc_guardian_rank).source_rank.sum()

5948

In [97]:
RESULT.append(5948)

#### Часть 2 

Теперь попробуем модифицировать метрику, чтобы она по-прежнему работала с ранками, но учитывала множественные вхождения документов в одну выдачу.

Для этого нам нужна сумма позиций. Но если сделать "в лоб", то тогда два документа с позицией 2 и 3 будут равносильны одному документу на позиции 5. Чтобы такого эффекта не было, нужно инвертировать ранки по формуле **11 - rank**. Тогда документ на последней десятой позиции получит 1 очко, документ на первой позиции получит 10. Если же в выдаче не было вообще ни одного вхождения, результатом будем считать 0 очков. И уже тут чем показатель больше, тем лучше. Один документ на 5 позиции получит 6 очков, а два документа на позициях 2 и 3 получат 9 + 8 = 17 очков.

Метрика – сумма по всем парам, как и в предыдущем варианте. Полученное значение нужно записать во вторую строчку выходного файла.

In [60]:
def calc_source_acc_rank(series: pd.Series, source: str) -> pd.Series:
    ranks = MAX_DEFAULT_RANK - series[series['displayLink'] == source]['rank'].values
    return pd.Series({'acc_rank' : ranks.sum()}, index=['acc_rank'])

calc_guardian_acc_rank = partial(calc_source_acc_rank, source='www.theguardian.com')

In [61]:
serp_clubs.groupby(['searchTerms', 'hl']).apply(calc_guardian_acc_rank).acc_rank.sum()

149

In [98]:
RESULT.append(149)

#### Часть 3

На абсолютные значения метрик смотреть можно, но понимания относительно других хостов они понимания не дают. Давайте посчитаем, насколько хорош самый хороший сайт по метрике "максимальное значение **11 - rank**". Если в предыдущей части мы брали сумму таких документов, то теперь хотим смотреть только на позицию лучшего.

Напишите в третью строчку домен displayLink, который имеет наибольшее значение этой метрики.

In [76]:
def calc_source_best_rank(series: pd.Series, source: str) -> pd.Series:
    ranks = MAX_DEFAULT_RANK - series[series['displayLink'] == source]['rank'].values
    max_rank = max(ranks) if len(ranks) else 0
    return pd.Series({'best_rank' : max_rank}, index=['best_rank'])

In [77]:
displaylinks = serp_clubs['displayLink'].unique()

Посчитаем честно, хотя результат +- предсказуем

In [83]:
acc_rank_values = []

serp_clubs_gb = serp_clubs.groupby(['searchTerms', 'hl'])

for i, link in enumerate(displaylinks):
    print('\r', end=f'{i}/{len(displaylinks)}, {link}')  # don't want to install tqdm
    part_func = partial(calc_source_best_rank, source=link)
    val = serp_clubs_gb.apply(part_func).best_rank.sum()
    acc_rank_values.append((link, val))

875/876, www.elfvoetbal.nlvergelijken.nlrt

In [88]:
from operator import itemgetter

sorted(acc_rank_values, key=itemgetter(1), reverse=True)

[('en.wikipedia.org', 2526),
 ('www.sofascore.com', 1171),
 ('pt.wikipedia.org', 770),
 ('nl.wikipedia.org', 758),
 ('it.wikipedia.org', 756),
 ('es.wikipedia.org', 730),
 ('de.wikipedia.org', 729),
 ('us.soccerway.com', 637),
 ('fr.wikipedia.org', 564),
 ('www.transfermarkt.de', 536),
 ('www.vi.nl', 469),
 ('www.livescore.cz', 407),
 ('www.transfermarkt.it', 381),
 ('www.espn.com.br', 314),
 ('globoesporte.globo.com', 282),
 ('www.futbol24.com', 262),
 ('www.newsnow.co.uk', 260),
 ('www.youtube.com', 250),
 ('www.bbc.co.uk', 229),
 ('www.futebol.com', 211),
 ('www.nieuwsblad.be', 206),
 ('www.ran.de', 196),
 ('www.bbc.com', 195),
 ('twitter.com', 187),
 ('www.kicker.de', 184),
 ('www.sport1.de', 183),
 ('espndeportes.espn.com', 178),
 ('www.whoscored.com', 177),
 ('www.marca.com', 174),
 ('www.espn.com', 171),
 ('www.zerozero.pt', 166),
 ('www.infobetting.com', 155),
 ('www.matchendirect.fr', 154),
 ('www.facebook.com', 143),
 ('www.livefoot.fr', 143),
 ('www.ad.nl', 141),
 ('www.theg

In [99]:
RESULT.append('en.wikipedia.org')

---

In [100]:
RESULT

[5948, 149, 'en.wikipedia.org']

In [110]:
with open('results.tsv', 'w') as f:
    for item in RESULT:
        f.write(f'{item}\n')